In [2]:
# 구글 드라이브 연결

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/examples/5장/data'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())
os.listdir(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 작업 디렉토리: /content/drive/MyDrive/examples/5장/data


['shape_predictor_68_face_landmarks.dat.bz2',
 'shape_predictor_68_face_landmarks.dat.bz2.5',
 'shape_predictor_68_face_landmarks.dat.bz2.3',
 'shape_predictor_68_face_landmarks.dat.bz2.1',
 'shape_predictor_68_face_landmarks.dat.bz2.2',
 'shape_predictor_68_face_landmarks.dat.bz2.4',
 'shape_predictor_68_face_landmarks.dat.bz2.6',
 'shape_predictor_68_face_landmarks.dat',
 'sample.mov',
 'video_musk.mp4',
 'elon_deep.mp4']

In [ ]:
!pip install opencv-python dlib numpy
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2024-10-27 08:05:27--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  63.8MB/s    in 1.0s    

2024-10-27 08:05:28 (63.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
from google.colab.patches import cv2_imshow

# 눈의 깜빡임 비율 계산 함수 (EAR: Eye Aspect Ratio)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# 눈 감지 임계값 및 프레임 길이 설정
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3

# dlib의 얼굴 탐지기와 랜드마크 예측기 로드
detector = dlib.get_frontal_face_detector()

# Colab에 랜드마크 예측기 파일 다운로드 (한 번만 실행 필요)
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# 파일 경로 설정
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 눈의 랜드마크 인덱스 (좌측 및 우측 눈)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)

# 파일 경로 설정 (Google Colab에서 업로드한 파일 경로 사용)
file_path = 'sample.mov'  # Colab의 '/content/' 디렉토리에 업로드한 영상 파일

# 비디오 스트림 시작 (파일에서 읽기)
cap = cv2.VideoCapture(file_path)

if not cap.isOpened():
    print("Error: Video file not opened. Check the file path or file format.")
else:
    print("Video file successfully opened.")

blink_count = 0
frame_count = 0
total_frames = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished reading the video or no frame captured.")
        break

    total_frames += 1
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = detector(gray, 0)

    if len(faces) == 0:
        print("No faces detected in this frame.")

    for face in faces:
        shape = predictor(gray, face)
        shape = np.array([[p.x, p.y] for p in shape.parts()])

        # 좌측 및 우측 눈 좌표 추출
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        # EAR 계산
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # 눈 깜빡임 감지
        if ear < EYE_AR_THRESH:
            frame_count += 1
        else:
            if frame_count >= EYE_AR_CONSEC_FRAMES:
                blink_count += 1
            frame_count = 0

        # 눈 영역을 그리기
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # EAR 및 깜빡임 횟수 표시
        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Blinks: {blink_count}", (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Google Colab에서 이미지를 표시
    cv2_imshow(frame)

# 비디오 스트림 해제
cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 비정상적인 깜빡임 패턴 여부 판단
blink_rate = blink_count / (total_frames / 30)  # 깜빡임 빈도 계산 (평균 초당 깜빡임 횟수)
print(f"Blink Rate: {blink_rate:.2f} blinks per second")

# 정상적인 범위 설정 (예: 0.25 ~ 0.5 blink/second)
if blink_rate < 0.15 or blink_rate > 0.5:
    print("Warning: Abnormal blinking pattern detected. The video might be a deepfake.")
else:
    print("Blinking pattern appears normal.")

Blink Rate: 0.54 blinks per second


In [ ]:
# 얼굴 경계선 정보 판단
mean_intensity_diff = np.mean(intensity_diffs)
mean_texture_diff = np.mean(texture_diffs)

print(f"Mean Intensity Difference: {mean_intensity_diff:.2f}")
print(f"Mean Texture Difference: {mean_texture_diff:.2f}")

# 특정 임계값 설정 (임계값은 실험적 데이터 기반으로 설정해야 함)
if mean_intensity_diff > 10 or mean_texture_diff > 0.2:
    print("Warning: Abnormal boundary characteristics detected. The video might be a deepfake.")
else:
    print("Boundary characteristics appear normal.")

In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
from google.colab.patches import cv2_imshow

# 눈의 깜빡임 비율 계산 함수 (EAR: Eye Aspect Ratio)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# 얼굴 경계선 내외부 밝기 및 색상 차이 계산 함수
def calculate_intensity_difference(face, frame, margin=10):
    # 얼굴 영역 마스크 생성
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.rectangle(mask, (face.left(), face.top()), (face.right(), face.bottom()), 255, -1)

    # 내부와 외부 영역 정의
    internal_region = mask.copy()
    external_region = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_RECT, (2*margin, 2*margin)))

    # 내부와 외부 밝기 계산
    internal_mean = cv2.mean(frame, mask=internal_region)[0]
    external_mean = cv2.mean(frame, mask=external_region - internal_region)[0]

    return abs(internal_mean - external_mean)

# 눈 감지 임계값 및 프레임 길이 설정
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3

# dlib의 얼굴 탐지기와 랜드마크 예측기 로드
detector = dlib.get_frontal_face_detector()

# 파일 경로 설정
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 눈의 랜드마크 인덱스 (좌측 및 우측 눈)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)

# 파일 경로 설정 (Google Colab에서 업로드한 파일 경로 사용)
file_path = 'sample.mov'

# 비디오 스트림 시작 (파일에서 읽기)
cap = cv2.VideoCapture(file_path)

if not cap.isOpened():
    print("Error: Video file not opened. Check the file path or file format.")
else:
    print("Video file successfully opened.")

blink_count = 0
frame_count = 0
total_frames = 0
intensity_diffs = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished reading the video or no frame captured.")
        break

    if frame is None or frame.size == 0:
        print("Warning: Empty or NoneType frame detected. Skipping this frame.")
        continue

    total_frames += 1
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = detector(gray, 0)

    if len(faces) == 0:
        print(f"No faces detected in frame {total_frames}.")
        continue

    for face in faces:
        shape = predictor(gray, face)
        shape = np.array([[p.x, p.y] for p in shape.parts()])

        # 좌측 및 우측 눈 좌표 추출
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        # EAR 계산
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # 눈 깜빡임 감지
        if ear < EYE_AR_THRESH:
            frame_count += 1
        else:
            if frame_count >= EYE_AR_CONSEC_FRAMES:
                blink_count += 1
            frame_count = 0

        # 얼굴 경계선 내외부 밝기 차이 계산
        intensity_diff = calculate_intensity_difference(face, gray)
        intensity_diffs.append(intensity_diff)

        # 눈 영역을 그리기
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # EAR 및 깜빡임 횟수 표시
        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Blinks: {blink_count}", (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Google Colab에서 이미지를 표시
    cv2_imshow(frame)  # 필요 시 표시

# 비디오 스트림 해제
cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 비정상적인 깜빡임 패턴 여부 판단
blink_rate = blink_count / (total_frames / 30)  # 깜빡임 빈도 계산 (평균 초당 깜빡임 횟수)
print(f"Blink Rate: {blink_rate:.2f} blinks per second")

# 정상적인 범위 설정 (예: 0.15 ~ 0.5 blink/second)
if blink_rate < 0.15 or blink_rate > 0.5:
    print("Warning: Abnormal blinking pattern detected. The video might be a deepfake.")
else:
    print("Blinking pattern appears normal.")


Blink Rate: 0.54 blinks per second


In [ ]:
# 얼굴 경계선 정보 판단
mean_intensity_diff = np.mean(intensity_diffs)

print(f"Mean Intensity Difference: {mean_intensity_diff:.2f}")

# 특정 임계값 설정 (임계값은 실험적 데이터 기반으로 설정해야 함)
if mean_intensity_diff > 10:  # 임계값 조정 가능
    print("Warning: Abnormal boundary characteristics detected. The video might be a deepfake.")
else:
    print("Boundary characteristics appear normal.")


Mean Intensity Difference: 45.18
